## iPython notebook for conversational retrieval using Langchain, OpenAI (API key required), and existing Pinecone index (API key required).
No need to ingest the documents/texts in this file; it is assumed that they have already beein ingested and the embeddings have already been stored in Pinecone with an index

In [ ]:
#!pip3 install langchain pinecone-client	# install if needed

* Open AI API key (from .bashrc, Windows environment variables, etc. Or .env) and embeddings

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
import os

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

* Use Open AI LLM with gpt-3.5-turbo. Set the temperature to be 0 if you do not want it to make up things

In [2]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)

* Set up Pinecone env

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# initialize pinecone
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

* Load the pre-created Pinecone index

In [4]:
# the index which has already be stored in pinecone.io as long-term memory
index_name = "langchaints"	# example pinecone index; replace by yours
if index_name in pinecone.list_indexes():
    docsearch = Pinecone.from_existing_index(index_name, embeddings)
else:
	raise ValueError('Cannot find the specified Pinecone index. Create one in pinecone.io or using pinecone.create_index(name=index_name, dimension=1536, metric="cosine", shards=1)')
# number of sources (split-documents when ingesting files); default is 4
k = 20
retriever = docsearch.as_retriever(
    search_type="similarity", search_kwargs={"k": 20})


* Set up to use the ConversationalRetrievalChain

In [13]:
from langchain.chains import ConversationalRetrievalChain
CRqa = ConversationalRetrievalChain.from_llm(
    llm, retriever=retriever, return_source_documents=True)

* Initialize chat history. Provide a prompt, search the index for similar texts (from the split-documents), and generate a reply based on only the found texts. Print out the reply and one source.

In [ ]:
chat_history = []
query = "What is the summary of the documents on CSI-RS?"
result = CRqa({"question": query, "chat_history": chat_history})
print(result['answer'])
# print out the first (most relevant) source (only the first 200 characters for brevity here) if needed
# print(result['source_documents'][0].page_content[:200])

* Update the chat history, and provide another prompt. Generate a reply and also print out one source.

In [ ]:
chat_history = [(query, result["answer"])]
query = "Elaborate how it is used for SCell activation?"
result = CRqa({"question": query, "chat_history": chat_history})
print(result['answer'])
# print out the first (most relevant) source (only the first 200 characters for brevity here) if needed
# print(result['source_documents'][0].page_content[:200])